In [20]:
from langchain.llms import ctransformers
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import chroma, faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import GrobidParser

In [2]:
# other imports
from utils import show_files

# model

In [33]:

MODEL_PATH = '/Users/shoaib/Downloads/zephyr/zephyr-7b-alpha.Q8_0.gguf'
MODEL_FOLDER = '/Users/shoaib/Downloads/zephyr'
RESEARCH_PAPERS_FOLDER ='/Users/shoaib/Desktop/deep learning/research papers/'
FAISS_STORE_PATH = './vector store/'
CHROMA_STORE_PATH = './chroma store/'
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512 
config = {"max_new_tokens":4096,"context_length":4096}

# for root, dir,files in os.walk(MODEL_FOLDER):
    # print(files)


In [4]:

def load_model():
    # callback = CallbackManager([StreamingStdOutCallbackHandler()])
    model = ctransformers.CTransformers(
        model=MODEL_PATH,
        model_type='mistral',
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        config=config
    )

    return model

# llm = load_model()

# document loader

In [45]:
# load documents
# docs_loader = DirectoryLoader(RESEARCH_PAPERS_FOLDER, show_progress=True)

In [46]:
# docs = docs_loader.load()

100%|██████████| 31/31 [00:39<00:00,  1.28s/it]


# grobid starts


In [ ]:
loader = GenericLoader.from_filesystem(
    './input paper/',
    glob="*",
    suffixes=[".pdf"],
    parser=GrobidParser(segment_sentences=False),
    show_progress=True,
)
docs = loader.load()

# grobid ends


In [7]:
total_content_len = 0
for i in docs:
    total_content_len += len(i.page_content)
print(total_content_len)


1713506


# text splitting

In [8]:
txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [9]:
all_splits = txt_splitter.split_documents(docs)

In [10]:
print("total no of splits:",len(all_splits))
print("words in 1st split:",len(all_splits[0].page_content))
print("preview of 1st split:",all_splits[0].page_content)

total no of splits: 2368
words in 1st split: 336
preview of 1st split: 5 1 0 2

r p A 0 1

]

V C . s c [

6 v 6 5 5 1 . 9 0 4 1 : v i X r a

Published as a conference paper at ICLR 2015

VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION

Karen Simonyan∗ & Andrew Zisserman+ Visual Geometry Group, Department of Engineering Science, University of Oxford {karen,az}@robots.ox.ac.uk

ABSTRACT


In [11]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/e5-base-v2')

In [19]:
# vector_store = chroma.Chroma.from_documents(documents=all_splits,embedding=embeddings,persist_directory=VECTOR_STORE_PATH)
vector_store = faiss.FAISS.from_documents(documents=all_splits,embedding=embeddings)
vector_store.save_local(VECTOR_STORE_PATH)

# retriver

In [50]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What is a gan architecture?")


In [51]:
len(retrieved_docs)
print(retrieved_docs[0].page_content)

Because of its nature with set-level supervision, the problem can be naturally formulated using the framework of GANs which learns the embedding of the input samples and generates output samples locating within the subspace spanned by training samples. A GAN model often consists of a discriminator D and a generator G. The framework has been used for addressing the image-to-image translation problem which transforms an input image from the source domain X to the output image in the target domain Y [13].

6307

DY, D'Y

DY, D'Y

DX, D'X

DY

DY

DX

Y'

Y

X

Y'

Y

y t + n e d

GX

GX

X

G'Y

X''

G'X

Y''

I

X

consistency

consistency

(a) 1-way GAN

(b) 2-way GAN

Figure 1. The network architectures of 1-way and 2-way GANs.

In our application, the source domain X represents original images while the target domain Y contains images with the desired characteristics.
